<a href="https://colab.research.google.com/github/shiwoei-ai/ai_tech_and_biz_app_course/blob/main/workshop_3B_r2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 遷移模型實作
# 透過遷移式學習進行八哥辨視
# 本程式碼參考政大蔡炎龍教授「少年Py的大冒險：
# 成為Python AI深度學習達人的第一門課」書中案例。
# 本課程陳煒翔助教協助修改程式。

# ==================================================
# 1. 程式執行環境的準備
# 載入必要套件

# 載入標準數據分析、畫圖套件
# 載入可以協助清楚呈現與評估分類模型表現的套件

# 載入標準數據分析、畫圖、檔案處理等基礎套件
# - numpy/pandas：資料處理的基本工具。pandas 在本例中未直接
#   使用，但保留 import 可維持與原教材/環境一致。
# - matplotlib：視覺化（畫圖、顯示結果、中文字體設定等）。
# - os/zipfile：處理檔案路徑、解壓縮資料集。
# - 在 Colab 上存取 /content/ 目錄最直觀。

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile

# 載入可以協助清楚呈現與評估分類模型表現的套件：
# - confusion_matrix：用矩陣形式檢視各類別的預測/真實值對照。
# - classification_report：輸出 precision/recall/F1/支援度。
# - 在模型評估（Step 7）階段會派上用場。

from sklearn.metrics import confusion_matrix, classification_report


# 載入 TensorFlow Keras 所需的模組
# - ResNet50V2：本工作坊案例採用遷移學習（transfer learning）。
#   透過預訓練模型抽取通用影像特徵，再串接自己的分類頭（輸出層）。
# - Sequential/Dense：以序列式串接各層、加入最後的 Dense 層。
# - to_categorical：把標籤轉為 one-hot。
# - preprocess_input：ResNetV2 專屬資料前處理（將像素轉到 [-1,1]）。
# - load_img/img_to_array：載圖、轉成張量（H×W×C）。

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# 下載台北思源黑體（讓中文顯示更完整）
# - 在 Colab/某些環境，預設中文字體不足，圖表中文字可能亂碼。
# - 下載字型後，透過 matplotlib 設定預設字體，可確保中文正常顯示。
# - 若在無網路的環境，請改為掛載雲端硬碟或本地安裝字型。

!wget -O TaipeiSansTCBeta-Regular.ttf \
  "https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download"

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

# 將下載的字型註冊到 matplotlib，並設定為預設字型
fontManager.addfont("TaipeiSansTCBeta-Regular.ttf")
mpl.rc("font", family="Taipei Sans TC Beta")

In [ ]:
# ==================================================
# 2. 資料載入及檢查（Loading Data）

# 2.1 下載並解壓縮資料集
# - 這裡直接用 wget 取得 GitHub 上的 myna（八哥）資料集。
#   這是政大蔡炎龍教授準備的資料，他本身是方面的專家。
# - --no-check-certificate：忽略 SSL 憑證檢查（避免憑證問題）。
# - 下載後解壓縮到 /content/，便於後續用 os.listdir 逐檔讀取。
#
# 注意事項：
# - 本教學/示範資料很小（只有 23 張），主要目的在於流程示範。
#   真實商業應用需更多資料，並切分訓練/驗證/測試集避免過度擬合。

!wget --no-check-certificate \
  https://github.com/yenlung/Deep-Learning-Basics/raw/master/images/myna.zip \
  -O /content/myna.zip

local_zip = "/content/myna.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/content")
zip_ref.close()


# 2.2 讀取圖片並建立資料與標籤
# - 直接將三個資料夾視為三個類別（0/1/2）。
# - 以固定的 target_size=(256,256) 載入，確保輸入張量尺寸相同。
# - img_to_array 會得到形狀 (H, W, C) 的 numpy 陣列（dtype=uint8）。
# - 先把所有圖片堆疊到 data（list），標籤存到 target（list）。

base_dir = "/content/"

# 三類八哥對應的資料夾名稱（類別 0/1/2）
myna_folders = ["crested_myna", "javan_myna", "common_myna"]

# 以下標籤名稱主要用於後續顯示（包括混淆矩陣標軸、Gradio 介面等）
labels = [
    "(土)八哥 (Crested Myna)",
    "白尾八哥 (Javan Myna)",
    "家八哥 (Common Myna)"
]

data = []    # 存放影像資料（H×W×C）
target = []  # 存放對應類別整數標籤（0/1/2）

# 逐一走訪每個類別的資料夾
for i in range(3):
    thedir = base_dir + myna_folders[i]
    # 取得該資料夾裡所有檔名（可能含非影像檔，實務上可再加判斷）
    myna_fnames = os.listdir(thedir)

    for myna in myna_fnames:
        img_path = thedir + "/" + myna
        # 載入圖片並統一縮放成 256×256，以利模型 batch 化處理
        # 注意：此處未做資料清理（如壞檔、非影像），示範為主
        img = load_img(img_path, target_size=(256, 256))
        x = img_to_array(img)     # 轉成 numpy 陣列（像素 0–255）
        data.append(x)            # 加入影像資料
        target.append(i)          # 加入對應類別整數（0/1/2）

# 轉成 numpy 陣列，模型輸入需要連續記憶體與統一 dtype/shape
data = np.array(data)

# 檢視資料維度：(樣本數, 高, 寬, 通道)
# 例：(23, 256, 256, 3) 表示 23 張 256×256 的 RGB 影像
print("資料維度:", data.shape)

In [3]:
# ==================================================
# 3. 資料前處理
# 這裡分別針對輸入與輸出進行前處理，說明如下：
# 1) preprocess_input（ResNetV2 版本）：
#    - 會把像素從 [0,255] 按模型需求轉換到 [-1,1]。
#    - 遷移學習時，務必要使用與預訓練模型相容的前處理，
#      才能對齊特徵分布（否則效果大打折扣）。
#    - 既然別的專家（ResNetV2）已經幫我們把這件事做好了，
#      那我們就直接使用這些提供的與預訓練模型相容的前處理方法。
# 2) to_categorical：把整數標籤（0/1/2）轉為 one-hot，如 [1,0,0]。

x_train = preprocess_input(data)
y_train = to_categorical(target, 3)

print("第一筆資料的 One-Hot 編碼:", y_train[0])


第一筆資料的 One-Hot 編碼: [1. 0. 0.]


In [ ]:
# ==================================================
# 4. 建立模型（遷移模型）
#
# 我們在此宣告遷移學習模型的設計結構：
# - 使用 ResNet50V2（include_top=False 表示去掉 ImageNet 的 1000
#   類別分類頭），保留其卷積骨幹作為特徵抽取器。
# - pooling='avg'會把最後一層的 2048 個特徵圖（計分板）做平均，
#   得到一個通道維度向量，參數量小、可降低過度擬合風險。
# - 凍結參數（trainable=False）可避免在小數據上破壞預訓練權重。
# - 只訓練自己加上的 Dense(3, softmax) 層，等於訓練一個分類頭。

resnet = ResNet50V2(include_top=False, pooling="avg")

# 凍結骨幹網路權重以保留預訓練特徵（小數據情境特別重要）
resnet.trainable = False

# 以 Sequential 串接：
# 以 ResNet50V2 做為骨幹 + 自訂輸出層（3 類 softmax）
model = Sequential()
model.add(resnet)
model.add(Dense(3, activation="softmax"))

In [ ]:
# ==================================================
# 5. 編譯模型
# - loss：categorical_crossentropy（多類別的標準損失函數）
# - optimizer：adam（具有自適應學習率的優化器，收斂通常較穩定）
# - metrics：accuracy（模型訓練過程中展示直觀指標）
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# 檢視模型摘要：
# 可看到總參數數量（ResNet50V2 約有 2 千多萬個參數），
# 但目前已凍結，所以可訓練參數僅剩最後 Dense 層（約 6147）。
# 這對於本案例僅有 23 張訓練資料的情況很關鍵，
# 可以降低過度擬合與訓練不穩定的情況。
model.summary()

In [ ]:
# ==================================================
# 6. 訓練模型
# - 本案例使用全部 23 張資料作為訓練資料，同時用於評估（僅為
#   Demo）。真實專案務必切分 train/valid/test，或用交叉驗證。
#   否則很可能出現過度擬合而不自知。
# - batch_size=23：小數據可整批訓練，讓每個 epoch 僅一次權重更新。
# - epochs=10：示範用；若資料更多/需要更好表現可增加並加入早停。

print("開始訓練模型...")
model.fit(x_train, y_train, batch_size=23, epochs=10)
print("模型訓練完成！")

In [ ]:
# ==================================================
# 7. 評估模型
# 以相同資料做 evaluate
# 這僅是示範用，正式流程應使用留下來的測試集
loss, acc = model.evaluate(x_train, y_train)
print(f"\n最終損失 (Loss): {loss}")
print(f"最終準確率 (Accuracy): {acc}")


# 視覺化模型訓練表現
# - 用模型在 x_train 上的預測結果（因示範資料量極小）來繪製
#   混淆矩陣與分類報告，觀察各類別正確與誤判情形。
# - seaborn.heatmap 可視化混淆矩陣更直觀。
# 注意：這裡評估不代表泛化能力（即真實的預測能力）；
#       正式商業或學術應用專案需對 test set 作圖與報告。

import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# 取得每張圖的預測類別（argmax 將 softmax 機率轉為 0/1/2）
y_predict = np.argmax(model.predict(x_train), axis=-1)

# 混淆矩陣：列（真實）× 欄（預測）
mat = confusion_matrix(target, y_predict)

plt.figure(figsize=(8, 6))
sns.heatmap(
    mat, square=True, annot=True, fmt="d", cbar=True, cmap="Blues",
    xticklabels=labels, yticklabels=labels
)
plt.title("Confusion Matrix (混淆矩陣)")
plt.ylabel("True label (真實標籤)")
plt.xlabel("Predicted label (預測標籤)")
plt.show()

In [ ]:
# ==================================================
# 8.：立互動式辨識應用（Gradio）
# - 讓大家上傳一張八哥圖片，即時得到三類機率輸出。
# - 這是從「建模」走向「應用/互動」的重要一步。
# - 注意影像尺寸：本例訓練時用 256×256；Gradio 預設會回傳
#   一張 numpy 陣列（H×W×C），若需要固定大小，也許需在
#   gr.Image(...) 指定參數如 image_mode/shape 等。

!pip install -q gradio
import gradio as gr

# 定義預測函式：
# - 輸入 inp：一張影像（numpy 陣列，H×W×C，uint8）。
# - expand_dims：增加 batch 維度以符合 Keras 預期 (N×H×W×C)。
# - preprocess_input：套用 ResNetV2 前處理（轉 [-1,1]）。
# - model.predict --> flatten --> 轉成標籤:機率 的 dict 給 Gradio。
def classify_image(inp):
    inp = np.expand_dims(inp, axis=0)
    inp = preprocess_input(inp)
    prediction = model.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(3)}

# 準備 Gradio 介面元件
image_input = gr.Image(label="請上傳一張八哥照片")
label_output = gr.Label(
    num_top_classes=3, label="AI 辨識結果"
)

title = "AI 八哥辨識機"
description = (
    "我能辨識 (土)八哥、白尾八哥、及家八哥。"
    "請找一張八哥照片來考我吧！"
)

# 準備範例影像路徑清單（方便一鍵測試介面）
sample_images = []
for folder in myna_folders:
    thedir = os.path.join(base_dir, folder)
    for file in os.listdir(thedir):
        sample_images.append(os.path.join(thedir, file))

# 啟動 Gradio 介面
# - share=True：產生公開網址（教學演示很方便）。
# - debug=True：遇到錯誤時顯示詳細訊息，利於除錯。
gr.Interface(
    fn=classify_image,
    inputs=image_input,
    outputs=label_output,
    title=title,
    description=description,
    examples=sample_images
).launch(share=True, debug=True)
